In [3]:
# dataset: - https://www.kaggle.com/datasets/yasserh/uber-fares-dataset

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Although not explicitly used in the output, it was imported
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso # 'Rid' in the source is likely 'Ridge', and 'Lasso' was used later
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.impute import SimpleImputer

In [5]:

# Load the dataset
# Assuming 'uber.csv' is in the current directory
df = pd.read_csv('/content/sample_data/uber.csv')

# Display the head of the dataframe (as shown in Out[3])
df.head()


,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [6]:

# --- Pre-process the dataset ---

# Drop unnecessary columns
# The document shows an attempt to drop 'Unnamed: 0', 'key', and 'pickup_dat'
# The full column name for the last one is 'pickup_datetime' based on df.head()
df = df.drop(columns=['Unnamed: 0', 'key', 'pickup_datetime'], errors='ignore')

# Handle missing values
imputer = SimpleImputer(strategy='mean')
# Fit and transform the dataframe, preserving column names by creating a new DataFrame
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Split the data into features (X) and target (y)
X = df_imputed.drop(columns=['fare_amount'])
y = df_imputed['fare_amount']

# Split the data into training and testing sets (using default split of 75/25)
# Assuming a standard random state wasn't set, or we can omit it for a basic reproduction
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) # Added random_state for reproducibility

# Standardize the features (scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# --- Implement Regression Models ---

# Implement Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)

# Implement Ridge Regression (alpha=1.0 used in source)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Implement Lasso Regression (alpha=0.1 used in source)
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_model.predict(X_test_scaled)


In [8]:


# --- Evaluate the Models ---

# Evaluation function
def evaluate_model(y_true, y_pred, model_name):
    """Calculates and prints R2 Score and RMSE."""
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Formatting matches the output structure as closely as possible
    print(f"{model_name}")
    print(f"R2 Score: {r2:.4f}, RMSE: {rmse:.2f}")

# Evaluate and compare
evaluate_model(y_test, y_pred_lr, "Linear Regression")
evaluate_model(y_test, y_pred_ridge, "Ridge Regression")
evaluate_model(y_test, y_pred_lasso, "Lasso Regression")

Linear Regression
R2 Score: 0.0003, RMSE: 10.14
Ridge Regression
R2 Score: 0.0003, RMSE: 10.14
Lasso Regression
R2 Score: -0.0000, RMSE: 10.14
